In [ ]:
!chmod +x ./setup.sh
!./setup.sh

In [ ]:
import boto3, sagemaker, os
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

In [ ]:
MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
PUBLIC_ACCOUNT_ID = '763104351884'

REGION = boto3.session.Session().region_name

MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'
PUBLIC_ECR_ENDPOINT =  f'{PUBLIC_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'

IMAGE_NAME = 'lampps'
TAG=':latest'

MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

# # Public ECR ログイン
!aws ecr get-login-password --region ap-northeast-1 | docker login --username AWS --password-stdin {PUBLIC_ECR_ENDPOINT}

In [ ]:
%%time

%cd ./container
!docker stop $(docker ps -q)
!docker rm $(docker ps -q -a)
!docker rmi -f $(docker images -a -q)
!docker build -t {IMAGE_NAME}{TAG} .
%cd ../

In [ ]:
# !docker run --gpus all -v {os.getcwd()}/src/:/test -d {IMAGE_NAME}{TAG} sh -c 'tail -f /dev/null'
# print(f'docker run --gpus all -v {os.getcwd()}/src/:/test -it --rm --entrypoint "bash" lampps:latest')

In [ ]:
!$(aws ecr get-login --region {REGION} --registry-ids {MY_ACCOUNT_ID} --no-include-email)
 
# リポジトリの作成
# すでにある場合はこのコマンドは必要ない
!aws ecr delete-repository --repository-name {IMAGE_NAME} --force
!aws ecr create-repository --repository-name {IMAGE_NAME}
 
# !docker push $image_uri
!docker tag {IMAGE_NAME}{TAG} {MY_IMAGE_URI}
!docker push {MY_IMAGE_URI}

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

In [61]:
prefix = 'lammps_simple'
input_s3_uri = sagemaker.session.Session().upload_data(path='param/', key_prefix=prefix)
print(input_s3_uri)

s3://sagemaker-ap-northeast-1-155580384669/lammps_simple


In [62]:
# S3 から SageMaker Processing インスタンスへの転送先
PROCESSING_INPUT_DIR = '/opt/ml/processing/input/'

# SageMaker Processing インスタンス内の結果出力先
PROCESSING_OUTPUT_DIR = '/opt/ml/processing/output'

print(PROCESSING_INPUT_DIR)
print(PROCESSING_OUTPUT_DIR)

/opt/ml/processing/input/
/opt/ml/processing/output


In [63]:
processor = sagemaker.processing.ScriptProcessor(
    base_job_name='LAMMPS-polyethylene',
    image_uri=MY_IMAGE_URI,
    command=['python'],
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.g4dn.xlarge'
)

In [64]:
processor.run(code='src/run.py', # S3 の URI でも可
             inputs=[
                 ProcessingInput(source=input_s3_uri,destination=PROCESSING_INPUT_DIR),
             ],
             outputs=[
                 ProcessingOutput(output_name='result',source=PROCESSING_OUTPUT_DIR)],
              arguments=[
                  '--input-dir',         PROCESSING_INPUT_DIR,
                  '--input-equiliv-in',  'lmp_equiliv.in',
                  '--input-equiliv-sh',  'lmp_equiliv.sh',
                  '--input-lmp2data-py', 'lmp2data.py',
                  '--np',                '2',
                  '--gpu',               '1',
                  '--output-dir',        PROCESSING_OUTPUT_DIR,
              ]
             )


Job Name:  LAMMPS-polyethylene-2021-10-12-14-47-36-440
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/lammps_simple', 'LocalPath': '/opt/ml/processing/input/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-12-14-47-36-440/input/code/run.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-northeast-1-155580384669/LAMMPS-polyethylene-2021-10-12-14-47-36-440/output/result', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
.........................Received argument

UnexpectedStatusException: Error for Processing job LAMMPS-polyethylene-2021-10-12-14-47-36-440: Failed. Reason: AlgorithmError: See job logs for more information

In [ ]:
processor.latest_job.describe()